## Projet : Employee-Attrition

In [28]:
from pyspark.ml.classification import LogisticRegression, RandomForestClassifier, GBTClassifier,DecisionTreeClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator, RegressionEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.tuning import TrainValidationSplit, ParamGridBuilder
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml import Pipeline
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql.functions import stddev, avg

## chargement du dataset

In [29]:
spark = SparkSession.builder.appName('attrition').getOrCreate()
file_location = "HR-Employee-Attrition.csv"
file_type = "csv"
# CSV options
infer_schema = "false"
first_row_is_header = "true"
delimiter = ","
# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.format(file_type) \
.option("inferSchema", infer_schema) \
.option("header", first_row_is_header) \
.option("sep", delimiter) \
.load(file_location)

df = df.drop("Over18")
df = df.drop("EmployeeCount")
df = df.drop("StandardHours")

## grid pour les modeles

In [30]:
gbt = GBTClassifier(featuresCol='features', labelCol='label')
param_grid_gbt = ParamGridBuilder() \
    .addGrid(gbt.maxDepth, [2, 4, 6]) \
    .addGrid(gbt.maxBins, [20, 30]) \
    .addGrid(gbt.maxIter, [10, 20, 30]) \
    .build()

rf = RandomForestClassifier(featuresCol='features', labelCol='label')
param_grid_rf = ParamGridBuilder() \
    .addGrid(rf.maxDepth, [2]) \
    .addGrid(rf.maxBins, [20]) \
    .addGrid(rf.numTrees, [10]) \
    .build()

lr = LogisticRegression(featuresCol='features', labelCol='label')
param_grid_lr = ParamGridBuilder() \
    .addGrid(lr.regParam, [0.01, 0.05, 0.1]) \
    .addGrid(lr.elasticNetParam, [0.0, 0.5, 1.0]) \
    .build()

## fonctions

In [31]:
def convert_to_int(df):
    # convertir les colonnes string en double quand cela est possible
    for column in df.columns:
        try:
            if int(df.select(column).first()[0]):
                df = df.withColumn(column, col(column).cast("double"))
        except:
            pass
    return df
        
def remove_outliers(df):

    # Boucle pour enlever les valeurs aberrantes de chaque colonne
    for col in df.columns:
        # Calcul des statistiques pour la colonne
        stats = df.select(avg(col), stddev(col)).first()
        mean = stats[0]
        std = stats[1]  
        if mean is not None and std is not None:
            # Calcul du seuil pour déterminer les valeurs aberrantes
            threshold = 3 * std + mean     
            # Suppression des valeurs aberrantes pour la colonne
            df = df.filter(df[col] <= threshold)
    return df


def assembler_for_pipeline(df, label):
    categoricalColumns = [col for (col, dtype) in df.dtypes if dtype == "string" and col != label]
    stages = []

    for categoricalCol in categoricalColumns:
        stringIndexer = StringIndexer(inputCol = categoricalCol, outputCol = categoricalCol + 'Index')
        encoder = OneHotEncoder(inputCols=[stringIndexer.getOutputCol()], outputCols=[categoricalCol + "classVec"])
        stages += [stringIndexer, encoder]
    
    label_stringIdx = StringIndexer(inputCol=label, outputCol="label")
    stages += [label_stringIdx]

    numericCols = [col for (col, dtype) in df.dtypes if dtype.startswith('double') and col != "label"]
    assemblerInputs = [c + "classVec" for c in categoricalColumns] + numericCols
    assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")
    return assembler, stages

def model_for_pipeline(classifier, param_grid):
    evaluator = BinaryClassificationEvaluator(labelCol='label')
    cv = CrossValidator(estimator=classifier, estimatorParamMaps=param_grid, evaluator=evaluator, numFolds=3)
    return cv

def view_feature_importances(df):
    
    assembler, stages = assembler_for_pipeline(df, 'Attrition')
    rf = RandomForestClassifier(labelCol="label", featuresCol="features")
    paramGrid = ParamGridBuilder() \
        .addGrid(rf.numTrees, [10, 20]) \
        .addGrid(rf.maxDepth, [5, 10]) \
        .build()
    
    evaluator = BinaryClassificationEvaluator(labelCol="label", rawPredictionCol="rawPrediction", metricName="areaUnderROC")
    train, test = df.randomSplit([0.8, 0.2])
    stages += [assembler, rf]
    pipeline = Pipeline(stages=stages)
    
    pipelineModel = pipeline.fit(train)
    predictions = pipelineModel.transform(test)
    auc = evaluator.evaluate(predictions)
    print("AUC-ROC = %g" % auc)
    
    importances = pipelineModel.stages[-1].featureImportances.toArray()
    cols = df.columns
    selectedcols = ["label", "features"] + cols
    cols_importances = list(zip(selectedcols, importances))
    sorted_cols_importances = sorted(cols_importances, key=lambda x: x[1], reverse=True)
    print("Colonnes triées par ordre d'importance décroissante :")
    for col, importance in sorted_cols_importances:
        print(col, "=", importance)
        
    return sorted_cols_importances
        
def pipelinedata(df, assembler, stages, model):
    
    stages += [assembler, model]
    train, test = df.randomSplit([0.8, 0.2])
    pipeline = Pipeline(stages=stages)
    pipelineModel = pipeline.fit(train)
    predictions = pipelineModel.transform(test)
    predictions.take(1)
    selected = predictions.select("label", "prediction", "rawPrediction", "probability")
    display(selected)
    return model, predictions, selected

def metrics(predictions, model):

    evaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction", labelCol="label", metricName="areaUnderROC")
    auc_roc = evaluator.evaluate(predictions)
    evaluator = MulticlassClassificationEvaluator(predictionCol="prediction", labelCol="label", metricName="accuracy")
    accuracy = evaluator.evaluate(predictions)
    evaluator = RegressionEvaluator(predictionCol="prediction", labelCol="label", metricName="rmse")
    rmse = evaluator.evaluate(predictions)

    print(model + ": courbe ROC = %g" % auc_roc)
    print(model + ": précision = %g" % accuracy)
    print(model + ": RMSE = %g" % rmse)
    
def filter_most_important_columns(df, sorted_cols_importances):
    selected_cols = [col for col, importance in sorted_cols_importances if importance > 0.01]
    if "features" in selected_cols:
        selected_cols.remove("features")
    if "label" in selected_cols:
        selected_cols.remove("label")
    df = df.select(selected_cols)
    return df

## Nettoyage

In [32]:
df = convert_to_int(df)
df = remove_outliers(df)

## Analyse de l'importance des colonnes

In [33]:
sorted_cols_importances = view_feature_importances(df)

AUC-ROC = 0.724248
Colonnes triées par ordre d'importance décroissante :
MonthlyRate = 0.13685865373538764
NumCompaniesWorked = 0.032945466495976
MonthlyIncome = 0.025491215407781563
OverTime = 0.018201824137640574
features = 0.016385920305205134
JobLevel = 0.013893615360948667
Attrition = 0.012996559215915976
YearsWithCurrManager = 0.008102447779980333
DistanceFromHome = 0.005671837218922965
WorkLifeBalance = 0.005042907983606777
PerformanceRating = 0.004990846427949578
YearsInCurrentRole = 0.0048987232040918465
BusinessTravel = 0.004644320539659293
MaritalStatus = 0.004205692609634758
Education = 0.0035786602838141874
YearsSinceLastPromotion = 0.0035647627074464443
TotalWorkingYears = 0.0034642039284802574
Department = 0.0034251160246222844
Gender = 0.00335503866629112
Age = 0.0029518212842760947
EducationField = 0.002116198314833574
DailyRate = 0.001938849577271772
JobSatisfaction = 0.0018806938462507493
label = 0.0018618002247707609
HourlyRate = 0.001236978023191154
YearsAtCompany 

## Garder les colonnes les plus importantes

In [34]:
df = filter_most_important_columns(df, sorted_cols_importances)

In [35]:
df.show(10)

+-----------+------------------+-------------+--------+--------+---------+
|MonthlyRate|NumCompaniesWorked|MonthlyIncome|OverTime|JobLevel|Attrition|
+-----------+------------------+-------------+--------+--------+---------+
|    19479.0|               8.0|       5993.0|     Yes|     2.0|      Yes|
|    24907.0|               1.0|       5130.0|      No|     2.0|       No|
|     2396.0|               6.0|       2090.0|     Yes|     1.0|      Yes|
|    23159.0|               1.0|       2909.0|     Yes|     1.0|       No|
|    16632.0|               9.0|       3468.0|      No|     1.0|       No|
|    11864.0|               0.0|       3068.0|      No|     1.0|       No|
|     9964.0|               4.0|       2670.0|     Yes|     1.0|       No|
|    13335.0|               1.0|       2693.0|      No|     1.0|       No|
|     8787.0|               0.0|       9526.0|      No|     3.0|       No|
|    16577.0|               6.0|       5237.0|      No|     2.0|       No|
+-----------+------------

## Modelisation avec les champs filtrés

In [36]:
df_gbt = df
df_rf = df
df_lr = df

### random forest

In [37]:
df_rf = convert_to_int(df_rf)
df_rf = remove_outliers(df_rf)

In [38]:
assembler, stages = assembler_for_pipeline(df_rf, 'Attrition')

In [39]:
model_rf = model_for_pipeline(rf, param_grid_rf)
model_rf, predictions_rf, selected_rf = pipelinedata(df_rf, assembler, stages, model_rf)
metrics(predictions_rf, 'rf')

DataFrame[label: double, prediction: double, rawPrediction: vector, probability: vector]

rf: courbe ROC = 0.743796
rf: précision = 0.843137
rf: RMSE = 0.396059


In [40]:
model_rf

CrossValidator_c4df840f5f10

### gradient boosting

In [98]:
df_gbt = convert_to_int(df_gbt)
df_gbt = remove_outliers(df_gbt)

In [100]:
df_gbt = convert_to_int(df_gbt)
df_gbt = remove_outliers(df_gbt)

assembler, stages = assembler_for_pipeline(df_gbt, 'Attrition')

model_gbt = model_for_pipeline(gbt, param_grid_gbt)
model_gbt, predictions_gbt, selected_gbt = pipelinedata(df_gbt, assembler, stages, model_gbt)
metrics(predictions_gbt, 'gbt')

In [101]:
model_gbt = model_for_pipeline(gbt, param_grid_gbt)
model_gbt, predictions_gbt, selected_gbt = pipelinedata(df_gbt, assembler, stages, model_gbt)
metrics(predictions_gbt, 'gbt')

DataFrame[label: double, prediction: double, rawPrediction: vector, probability: vector]

gbt: courbe ROC = 0.795025
gbt: précision = 0.828685
gbt: RMSE = 0.413902


In [11]:
model_gbt

NameError: name 'model_gbt' is not defined

### logistic regression

In [105]:
df_lr = convert_to_int(df_lr)
df_lr = remove_outliers(df_lr)

In [106]:
assembler, stages = assembler_for_pipeline(df_lr, 'Attrition')

In [107]:
model_lr = model_for_pipeline(lr, param_grid_lr)
model_lr, predictions_lr, selected_lr = pipelinedata(df_lr, assembler, stages, model_lr)
metrics(predictions_lr, 'lr')

DataFrame[label: double, prediction: double, rawPrediction: vector, probability: vector]

lr: courbe ROC = 0.76598
lr: précision = 0.818898
lr: RMSE = 0.425561


In [10]:
model_lr

NameError: name 'model_lr' is not defined

### j'ai pas utilisé cette partie, je te laisse modifié si besoin

In [61]:
# #paramétres
# def params(nom, nb_model, train, test):
#     liste = range(1, 20)

#     best_accuracy = 0
#     best_nb = 0

#     for nb in liste:
#         model_choix = [LogisticRegression(labelCol = 'label', featuresCol = 'features', maxIter= nb),
#         DecisionTreeClassifier(labelCol="label", featuresCol="features", maxDepth=nb, seed =40),
#         GBTClassifier(labelCol="label", featuresCol="features", maxIter=nb, seed =40),
#         RandomForestClassifier(labelCol="label", featuresCol="features", numTrees=nb, seed =40)]

#         model = model_choix[nb_model]
#         trained_model = model.fit(train)
#         predictions = trained_model.transform(test)
#         evaluator = MulticlassClassificationEvaluator(predictionCol="prediction", labelCol="label", metricName="accuracy")
#         accuracy = evaluator.evaluate(predictions)
    
#         # meilleure précision
#         if accuracy > best_accuracy:
#             best_accuracy = accuracy
#             best_nb = nb
        
#     print("la meilleure précision pour " + nom + " :", best_accuracy)
#     print("le meilleur nombre pour paramétre " + nom + " :", best_nb)

In [44]:
model_rf

CrossValidator_c4df840f5f10

In [43]:
import shutil
model_rf.save("Model")

Py4JJavaError: An error occurred while calling o8794.save.
: java.lang.RuntimeException: java.io.FileNotFoundException: java.io.FileNotFoundException: HADOOP_HOME and hadoop.home.dir are unset. -see https://wiki.apache.org/hadoop/WindowsProblems
	at org.apache.hadoop.util.Shell.getWinUtilsPath(Shell.java:735)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:270)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:286)
	at org.apache.hadoop.fs.RawLocalFileSystem.setPermission(RawLocalFileSystem.java:978)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkOneDirWithMode(RawLocalFileSystem.java:660)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermission(RawLocalFileSystem.java:700)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirs(RawLocalFileSystem.java:672)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermission(RawLocalFileSystem.java:699)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirs(RawLocalFileSystem.jav

In [ ]:
shutil.make_archive('Model', 'zip','./content/Model')

In [3]:
my_dict = {'rf': {'model': "test1", 'accuracy': 0.8181818181818182}, 'gbt': {'model': "test2", 'accuracy': 0.8284671532846716}, 'lr': {'model': "test2", 'accuracy': 0.8513011152416357}}

In [4]:
best_model = None
best_accuracy = 0

for model_name, model_info in my_dict.items():
    accuracy = model_info['accuracy']
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_model = model_info['model']

print(f"The best model is {best_model} with an accuracy of {best_accuracy}.")

The best model is test2 with an accuracy of 0.8513011152416357.
